In [2]:
import os
import torch.utils.data
from torch.nn import DataParallel
from datetime import datetime
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
#from config import BATCH_SIZE, PROPOSAL_NUM, SAVE_FREQ, LR, WD, resume, save_dir
#from core import model, dataset
#from core.utils import init_log, progress_bar

In [10]:
BATCH_SIZE = 16
PROPOSAL_NUM = 6
CAT_NUM = 4
INPUT_SIZE = (224, 224)  # (w, h)
LR = 0.001
WD = 1e-4
SAVE_FREQ = 1
resume = ''
test_model = 'model.ckpt'
save_dir = '/model'

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [11]:
x_train=np.load('x_train.npy')
y_train=np.load('y_train.npy')
x_valid=np.load('x_valid.npy')
y_valid=np.load('y_valid.npy')

In [13]:
#x_train = np.array(x_train, dtype='float32')
#y_train = np.array(y_train, dtype='int64')
#x_valid = np.array(x_valid, dtype='float32')
#y_valid = np.array(y_valid, dtype='int64')

In [14]:
x_train.shape,y_train.shape

((1140, 512, 512, 3), (1140, 1))

In [7]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(), # augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './data_10'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'Training'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'Test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

In [8]:
print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

학습 데이터셋 크기: 1140
테스트 데이터셋 크기: 236
클래스: ['30017_롯데밀키스250ML', '30033_코카콜라250ML', '40007_롯데마운틴듀250ML', '40036_오란씨깔라만시250ML', '40049_코카환타오렌지250ML', '50012_일화맥콜250ML', '60008_일화천연사이다250ML', '60009_롯데펩시콜라250ML', '60037_코카환타파인애플250ML', '90118_코카콜라)스프라이트250ML']


In [28]:
#anchor

import numpy as np
#from config import INPUT_SIZE

_default_anchors_setting = (#3부분?!!
    dict(layer='p3', stride=32, size=48, scale=[2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
    dict(layer='p4', stride=64, size=96, scale=[2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
    dict(layer='p5', stride=128, size=192, scale=[1, 2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
)


def generate_default_anchor_maps(anchors_setting=None, input_shape=INPUT_SIZE):
    """
    generate default anchor
    :param anchors_setting: all informations of anchors
    :param input_shape: shape of input images, e.g. (h, w)
    :return: center_anchors: # anchors * 4 (oy, ox, h, w)
             edge_anchors: # anchors * 4 (y0, x0, y1, x1)
             anchor_area: # anchors * 1 (area)
    """
    if anchors_setting is None:
        anchors_setting = _default_anchors_setting

    center_anchors = np.zeros((0, 4), dtype=np.float32)
    edge_anchors = np.zeros((0, 4), dtype=np.float32)
    anchor_areas = np.zeros((0,), dtype=np.float32)
    input_shape = np.array(input_shape, dtype=int)

    for anchor_info in anchors_setting:

        stride = anchor_info['stride']
        size = anchor_info['size']
        scales = anchor_info['scale']
        aspect_ratios = anchor_info['aspect_ratio']

        output_map_shape = np.ceil(input_shape.astype(np.float32) / stride) #np.ceil:소수점 올림!!->정수
        output_map_shape = output_map_shape.astype(np.int)
        output_shape = tuple(output_map_shape) + (4,)
        ostart = stride / 2.
        oy = np.arange(ostart, ostart + stride * output_shape[0], stride)
        oy = oy.reshape(output_shape[0], 1)
        ox = np.arange(ostart, ostart + stride * output_shape[1], stride)
        ox = ox.reshape(1, output_shape[1])
        center_anchor_map_template = np.zeros(output_shape, dtype=np.float32)
        center_anchor_map_template[:, :, 0] = oy
        center_anchor_map_template[:, :, 1] = ox
        for scale in scales:
            for aspect_ratio in aspect_ratios:
                center_anchor_map = center_anchor_map_template.copy()
                center_anchor_map[:, :, 2] = size * scale / float(aspect_ratio) ** 0.5
                center_anchor_map[:, :, 3] = size * scale * float(aspect_ratio) ** 0.5

                edge_anchor_map = np.concatenate((center_anchor_map[..., :2] - center_anchor_map[..., 2:4] / 2.,
                                                  center_anchor_map[..., :2] + center_anchor_map[..., 2:4] / 2.),
                                                 axis=-1)
                anchor_area_map = center_anchor_map[..., 2] * center_anchor_map[..., 3]
                center_anchors = np.concatenate((center_anchors, center_anchor_map.reshape(-1, 4)))
                edge_anchors = np.concatenate((edge_anchors, edge_anchor_map.reshape(-1, 4)))
                anchor_areas = np.concatenate((anchor_areas, anchor_area_map.reshape(-1)))

    return center_anchors, edge_anchors, anchor_areas

# IoU: https://deep-learning-study.tistory.com/402 
# predicted bounding box가 ground-truth bounding box와 얼마나 일치하는지 측정하기 위한 평가 지표를 정의
# area of overlap / area of union
def hard_nms(cdds, topn=10, iou_thresh=0.25): #non-maximum suppression
    #object detector가 예측한 bounding box 중에서 정확한 bounding box를 선택하도록 하는 기법
    #iou_thresh:한계점
    if not (type(cdds).__module__ == 'numpy' and len(cdds.shape) == 2 and cdds.shape[1] >= 5):
        raise TypeError('edge_box_map should be N * 5+ ndarray')

    cdds = cdds.copy()
    indices = np.argsort(cdds[:, 0])#모든 행 첫번째 열 #np.argsort:numpy array 정렬 #그래서 행이 총 3개면 [0,1,2]임
    cdds = cdds[indices]#cdds 그대로 가져옴 ###########이거 왜 필요하죠..? 그냥 cdds인데...
    cdd_results = []

    res = cdds

    while res.any():
        cdd = res[-1]# 마지막 행
        cdd_results.append(cdd)
        if len(cdd_results) == topn:
            return np.array(cdd_results)
        res = res[:-1] # 마지막 행 제외

        start_max = np.maximum(res[:, 1:3], cdd[1:3])# 두 개의 array에 대해 동일한 위치의 성분끼리 비교하여 최대값 또는 최소값 계산
        end_min = np.minimum(res[:, 3:5], cdd[3:5])
        lengths = end_min - start_max
        intersec_map = lengths[:, 0] * lengths[:, 1]
        intersec_map[np.logical_or(lengths[:, 0] < 0, lengths[:, 1] < 0)] = 0
        iou_map_cur = intersec_map / ((res[:, 3] - res[:, 1]) * (res[:, 4] - res[:, 2]) + (cdd[3] - cdd[1]) * (
            cdd[4] - cdd[2]) - intersec_map)
        res = res[iou_map_cur < iou_thresh]

    return np.array(cdd_results)


if __name__ == '__main__':
    a = hard_nms(np.array([
        [0.4, 1, 10, 12, 20],
        [0.5, 1, 11, 11, 20],
        [0.55, 20, 30, 40, 50]
    ]), topn=100, iou_thresh=0.4)
    print(a)

[[ 0.55 20.   30.   40.   50.  ]
 [ 0.5   1.   11.   11.   20.  ]]


In [12]:
########################

_default_anchors_setting = (#3부분?!!
    dict(layer='p3', stride=32, size=48, scale=[2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
    dict(layer='p4', stride=64, size=96, scale=[2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
    dict(layer='p5', stride=128, size=192, scale=[1, 2 ** (1. / 3.), 2 ** (2. / 3.)], aspect_ratio=[0.667, 1, 1.5]),
)
_default_anchors_setting 

({'layer': 'p3',
  'stride': 32,
  'size': 48,
  'scale': [1.2599210498948732, 1.5874010519681994],
  'aspect_ratio': [0.667, 1, 1.5]},
 {'layer': 'p4',
  'stride': 64,
  'size': 96,
  'scale': [1.2599210498948732, 1.5874010519681994],
  'aspect_ratio': [0.667, 1, 1.5]},
 {'layer': 'p5',
  'stride': 128,
  'size': 192,
  'scale': [1, 1.2599210498948732, 1.5874010519681994],
  'aspect_ratio': [0.667, 1, 1.5]})

In [14]:
#############################
INPUT_SIZE = (224, 224)
input_shape=INPUT_SIZE

center_anchors = np.zeros((0, 4), dtype=np.float32)
edge_anchors = np.zeros((0, 4), dtype=np.float32)
anchor_areas = np.zeros((0,), dtype=np.float32)
input_shape = np.array(input_shape, dtype=int)

center_anchors, edge_anchors, anchor_areas,input_shape 

(array([], shape=(0, 4), dtype=float32),
 array([], shape=(0, 4), dtype=float32),
 array([], dtype=float32),
 array([224, 224]))

In [15]:
##################################
stride = 32
size = 48
scales = [1.2599210498948732, 1.5874010519681994]
aspect_ratios = [0.667, 1, 1.5]

output_map_shape = np.ceil(input_shape.astype(np.float32) / stride) #np.ceil:소수점 올림!!->정수
output_map_shape = output_map_shape.astype(np.int)
output_shape = tuple(output_map_shape) + (4,)
ostart = stride / 2.
oy = np.arange(ostart, ostart + stride * output_shape[0], stride)
oy = oy.reshape(output_shape[0], 1)
ox = np.arange(ostart, ostart + stride * output_shape[1], stride)
ox = ox.reshape(1, output_shape[1])
center_anchor_map_template = np.zeros(output_shape, dtype=np.float32)
center_anchor_map_template[:, :, 0] = oy
center_anchor_map_template[:, :, 1] = ox
for scale in scales:
    for aspect_ratio in aspect_ratios:
        center_anchor_map = center_anchor_map_template.copy()
        center_anchor_map[:, :, 2] = size * scale / float(aspect_ratio) ** 0.5
        center_anchor_map[:, :, 3] = size * scale * float(aspect_ratio) ** 0.5

        edge_anchor_map = np.concatenate((center_anchor_map[..., :2] - center_anchor_map[..., 2:4] / 2.,
                                          center_anchor_map[..., :2] + center_anchor_map[..., 2:4] / 2.),
                                         axis=-1)
        anchor_area_map = center_anchor_map[..., 2] * center_anchor_map[..., 3]
        center_anchors = np.concatenate((center_anchors, center_anchor_map.reshape(-1, 4)))
        edge_anchors = np.concatenate((edge_anchors, edge_anchor_map.reshape(-1, 4)))
        anchor_areas = np.concatenate((anchor_areas, anchor_area_map.reshape(-1)))

In [20]:
output_map_shape,output_shape,oy,ox, center_anchor_map_template

(array([7, 7]),
 (7, 7, 4),
 array([[ 16.],
        [ 48.],
        [ 80.],
        [112.],
        [144.],
        [176.],
        [208.]]),
 array([[ 16.,  48.,  80., 112., 144., 176., 208.]]),
 array([[[ 16.,  16.,   0.,   0.],
         [ 16.,  48.,   0.,   0.],
         [ 16.,  80.,   0.,   0.],
         [ 16., 112.,   0.,   0.],
         [ 16., 144.,   0.,   0.],
         [ 16., 176.,   0.,   0.],
         [ 16., 208.,   0.,   0.]],
 
        [[ 48.,  16.,   0.,   0.],
         [ 48.,  48.,   0.,   0.],
         [ 48.,  80.,   0.,   0.],
         [ 48., 112.,   0.,   0.],
         [ 48., 144.,   0.,   0.],
         [ 48., 176.,   0.,   0.],
         [ 48., 208.,   0.,   0.]],
 
        [[ 80.,  16.,   0.,   0.],
         [ 80.,  48.,   0.,   0.],
         [ 80.,  80.,   0.,   0.],
         [ 80., 112.,   0.,   0.],
         [ 80., 144.,   0.,   0.],
         [ 80., 176.,   0.,   0.],
         [ 80., 208.,   0.,   0.]],
 
        [[112.,  16.,   0.,   0.],
         [112.,  48.,   0

In [23]:
center_anchors.shape, edge_anchors.shape, anchor_areas.shape

((294, 4), (294, 4), (294,))

In [24]:
center_anchors, edge_anchors, anchor_areas

(array([[ 16.      ,  16.      ,  74.049416,  49.39096 ],
        [ 16.      ,  48.      ,  74.049416,  49.39096 ],
        [ 16.      ,  80.      ,  74.049416,  49.39096 ],
        ...,
        [208.      , 144.      ,  62.21316 ,  93.31974 ],
        [208.      , 176.      ,  62.21316 ,  93.31974 ],
        [208.      , 208.      ,  62.21316 ,  93.31974 ]], dtype=float32),
 array([[-21.024708,  -8.69548 ,  53.024708,  40.69548 ],
        [-21.024708,  23.30452 ,  53.024708,  72.69548 ],
        [-21.024708,  55.30452 ,  53.024708, 104.69548 ],
        ...,
        [176.89342 ,  97.34013 , 239.10658 , 190.65987 ],
        [176.89342 , 129.34013 , 239.10658 , 222.65987 ],
        [176.89342 , 161.34013 , 239.10658 , 254.65987 ]], dtype=float32),
 array([3657.3718, 3657.3718, 3657.3718, 3657.3718, 3657.3718, 3657.3718,
        3657.3718, 3657.3718, 3657.3718, 3657.3718, 3657.3718, 3657.3718,
        3657.3718, 3657.3718, 3657.3718, 3657.3718, 3657.3718, 3657.3718,
        3657.3718, 365

In [ ]:
## https://deep-learning-study.tistory.com/403
## hard-nms,soft-nms 차이
## ^ https://eehoeskrap.tistory.com/407

import torch

from IoU import intersection_over_union

def nms(bboxes, iou_threshold, threshold, box_format='corners'):

    # bboxes가 list인지 확인합니다.
    assert type(bboxes) == list #뒤의 조건이 True가 아니면 assertError 나옴

    # box 점수가 threshold보다 높은 것을 선별합니다.
    # box shape는 [class, score, x1, y1, x2, y2] 입니다.
    bboxes = [box for box in bboxes if box[1] > threshold]
    # 점수 오름차순으로 정렬합니다.
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nmn = []

    # bboxes가 모두 제거될때 까지 반복합니다.
    while bboxes:
        # 0번째 index가 가장 높은 점수를 갖고있는 box입니다. 이것을 선택하고 bboxes에서 제거합니다.
        chosen_box = bboxes.pop(0)

        # box가 선택된 box와의 iou가 임계치보다 낮거나
        # class가 다르다면 bboxes에 남기고, 그 이외는 다 없앱니다.
        bboxes = [box for box in bboxes if box[0] != chosen_box[0] \
               or intersection_over_union(torch.tensor(chosen_box[2:]),
                                          torch.tensor(box[2:]),
                                          box_format=box_format)
                    < iou_threshold]

        # 선택된 박스를 추가합니다.
        bboxes_after_nmn.append(chosen_box)

    return bboxes_after_nmn


# IoU: https://deep-learning-study.tistory.com/402 
# predicted bounding box가 ground-truth bounding box와 얼마나 일치하는지 측정하기 위한 평가 지표를 정의
# area of overlap / area of union
def hard_nms(cdds, topn=10, iou_thresh=0.25): #non-maximum suppression
    #iou_thresh:한계점
    if not (type(cdds).__module__ == 'numpy' and len(cdds.shape) == 2 and cdds.shape[1] >= 5):
        raise TypeError('edge_box_map should be N * 5+ ndarray')

    cdds = cdds.copy()
    indices = np.argsort(cdds[:, 0])#모든 행 첫번째 열 #np.argsort:numpy array 정렬 #그래서 행이 총 3개면 [0,1,2]임
    cdds = cdds[indices] #cdds 그대로 가져옴 ###########이거 왜 필요하죠..? 그냥 cdds인데...
    cdd_results = []

    res = cdds

    while res.any():
        cdd = res[-1] #마지막 행
        cdd_results.append(cdd)
        if len(cdd_results) == topn:
            return np.array(cdd_results)
        res = res[:-1] #마지막 행 제외 

        start_max = np.maximum(res[:, 1:3], cdd[1:3])# 두 개의 array에 대해 동일한 위치의 성분끼리 비교하여 최대값 또는 최소값 계산
        end_min = np.minimum(res[:, 3:5], cdd[3:5])
        lengths = end_min - start_max
        intersec_map = lengths[:, 0] * lengths[:, 1]
        intersec_map[np.logical_or(lengths[:, 0] < 0, lengths[:, 1] < 0)] = 0
        iou_map_cur = intersec_map / ((res[:, 3] - res[:, 1]) * (res[:, 4] - res[:, 2]) + (cdd[3] - cdd[1]) * (
            cdd[4] - cdd[2]) - intersec_map)
        res = res[iou_map_cur < iou_thresh]

    return np.array(cdd_results)

In [63]:
c=np.array([
        [0.4, 0, 10, 12, 20],
        [0.5, 1, 11, 11, 20],
#        [0.55, 20, 30, 40, 50]
    ])

In [64]:
c[:,0]

array([0.4, 0.5])

In [65]:
indices=np.argsort(c[:,0])#[0,1,2]
np.argsort(c[:,0])

array([0, 1], dtype=int64)

In [66]:
c[indices]

array([[ 0.4,  0. , 10. , 12. , 20. ],
       [ 0.5,  1. , 11. , 11. , 20. ]])

In [68]:
#c[[0,1,2]]
c[[0,1]]

array([[ 0.4,  0. , 10. , 12. , 20. ],
       [ 0.5,  1. , 11. , 11. , 20. ]])

In [69]:
cdd=c[-1]
cdd

array([ 0.5,  1. , 11. , 11. , 20. ])

In [70]:
c.shape

(2, 5)

In [71]:
res=c[:-1]
res

array([[ 0.4,  0. , 10. , 12. , 20. ]])

In [72]:
start_max=np.maximum(res[:, 1:3], cdd[1:3])
start_max

array([[ 1., 11.]])

In [73]:
res[:,1:3]

array([[ 0., 10.]])

In [74]:
cdd[1:3]

array([ 1., 11.])

In [75]:
end_min=np.minimum(res[:, 3:5], cdd[3:5])
end_min

array([[11., 20.]])

In [76]:
lengths = end_min - start_max
lengths

array([[10.,  9.]])

In [77]:
intersec_map = lengths[:, 0] * lengths[:, 1]
intersec_map

array([90.])

In [78]:
intersec_map[np.logical_or(lengths[:, 0] < 0, lengths[:, 1] < 0)]

array([], dtype=float64)

In [79]:
np.logical_or(lengths[:, 0] < 0, lengths[:, 1] < 0)

array([False])

In [80]:
intersec_map[np.logical_or(lengths[:, 0] < 0, lengths[:, 1] < 0)]=0

In [81]:
intersec_map

array([90.])

In [82]:
iou_map_cur = intersec_map / ((res[:, 3] - res[:, 1]) * (res[:, 4] - res[:, 2]) + (cdd[3] - cdd[1]) * (
            cdd[4] - cdd[2]) - intersec_map)
iou_map_cur 

array([0.75])

In [83]:
res[iou_map_cur < 0.25]

array([], shape=(0, 5), dtype=float64)

In [84]:
#resnet

import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample# downsampling:차원을 줄여서 적은 메모리로 깊은 convolution을 할 수 있게 한다. 
        # 보통 stride를 2 이상으로 하는 convolution을 사용하거나 pooling을 사용한다. 
        # 이 과정을 거치면 어쩔 수 없이 feature의 정보를 잃게된다.
        # https://m.blog.naver.com/9709193/221979612209
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4
    # 1x1-> 3x3 -> 1x1
    
    # Convolution Parameters = Kernel Size x Kernel Size x Input Channel x Output Channel
    #1x1 Convolution: 연산량이 작기 때문에 Feature Map(Output Channel)을 줄이거나 키울 때 사용
    #차원, 채널 축소 후 공간적 특성 추출(3x3 Convolution이 연산량이 9배 많기 때문에), 채널 증가, parameters 감소


    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)# If True, adds a learnable bias to the output.
        ## BatchNorm에 bias가 포함되어 있으므로, conv2d는 bias=False로 설정합니다.
        ## https://deep-learning-study.tistory.com/534
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        feature1 = x
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = nn.Dropout(p=0.5)(x)
        feature2 = x
        x = self.fc(x)

        return x, feature1, feature2


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model